In [1]:
pip list

Package                   Version
------------------------- -----------
anyio                     4.0.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.2.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.0
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.1.0
certifi                   2023.7.22
cffi                      1.16.0
charset-normalizer        3.3.0
colorama                  0.4.6
comm                      0.1.4
contourpy                 1.1.0
customtkinter             5.2.2
cycler                    0.11.0
darkdetect                0.8.0
debugpy                   1.6.7.post1
decorator                 5.1.1
defusedxml                0.7.1
diffuser                  0.0.1
diffusers                 0.25.1
distlib                   0.3.7
et-xmlfile                1.1.0
executing                 1.2.0
fastjsonschema


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
archivo = 'Ejercicio Ingeniero de datos.xlsx'

df = pd.read_excel(archivo, sheet_name='Ejercicio 1', header = 4)
df = df.rename(columns={'2015p/': 2015})
df.head(5)

,Actividad económica,Entidad,Concepto,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Total de la actividad económica,Total nacional,Total nacional -Total de la actividad económica,1.255911e+07,1.305069e+07,1.334772e+07,1.393138e+07,1.425446e+07,1.440276e+07,1.364855e+07,1.435240e+07,1.487580e+07,1.543099e+07,1.564262e+07,1.606778e+07,1.657310e+07,1.702818e+07
1,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,1.211976e+05,1.265541e+05,1.296280e+05,1.381118e+05,1.503051e+05,1.509500e+05,1.432539e+05,1.522052e+05,1.589345e+05,1.677060e+05,1.728205e+05,1.920797e+05,1.979114e+05,2.167030e+05
2,Total de la actividad económica,Baja California,Baja California -Total de la actividad económica,3.995146e+05,4.230055e+05,4.330082e+05,4.560193e+05,4.615813e+05,4.575566e+05,4.077459e+05,4.281625e+05,4.407007e+05,4.560245e+05,4.655247e+05,4.745215e+05,5.060935e+05,5.244052e+05
3,Total de la actividad económica,Baja California Sur,Baja California Sur -Total de la actividad eco...,7.604759e+04,8.154601e+04,8.739767e+04,9.365562e+04,1.061994e+05,1.089754e+05,1.083388e+05,1.106564e+05,1.147078e+05,1.173458e+05,1.150276e+05,1.141693e+05,1.302958e+05,1.331467e+05
4,Total de la actividad económica,Campeche,Campeche -Total de la actividad económica,1.047511e+06,1.059561e+06,1.038534e+06,1.014280e+06,9.475755e+05,8.672310e+05,7.807574e+05,7.539686e+05,7.265039e+05,7.147871e+05,7.210851e+05,6.867465e+05,6.380826e+05,6.007712e+05


In [12]:
df = pd.melt(df, id_vars=['Actividad económica', 'Entidad','Concepto'], 
                    value_vars=[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
                    ignore_index=True,
                    var_name='Año', 
                    value_name='PIB')
df.head(5)

,Actividad económica,Entidad,Concepto,Año,PIB
0,Total de la actividad económica,Total nacional,Total nacional -Total de la actividad económica,2003,1.255911e+07
1,Total de la actividad económica,Aguascalientes,Aguascalientes -Total de la actividad económica,2003,1.211976e+05
2,Total de la actividad económica,Baja California,Baja California -Total de la actividad económica,2003,3.995146e+05
3,Total de la actividad económica,Baja California Sur,Baja California Sur -Total de la actividad eco...,2003,7.604759e+04
4,Total de la actividad económica,Campeche,Campeche -Total de la actividad económica,2003,1.047511e+06
